In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Char-RNN을 사용해 셰익스피어 문체 생성

In [2]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt',shakespeare_url)
with open(filepath) as f:
  shakespeare_txt = f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
type(shakespeare_txt)

str

In [ ]:
shakespeare_txt[1000:1200]

"Second Citizen:\nWould you proceed especially against Caius Marcius?\n\nAll:\nAgainst him first: he's a very dog to the commonalty.\n\nSecond Citizen:\nConsider you what services he has done for his country?"

In [ ]:
len(shakespeare_txt)

1115394

In [3]:
# 모든 글자를 정수로 인코딩, char_level=True로 지정하면 단어 수준 대신 글자 수준의 인코딩을 형성
# 이 클래스는 기본적으로 텍스트를 소문자로 바꾼다.(lower=False하면 이를 수행하지 않음)
tokenizer = keras.preprocessing.text.Tokenizer(char_level = True)
tokenizer.fit_on_texts(shakespeare_txt)

In [4]:
print('글자를 숫자로 : ',tokenizer.texts_to_sequences(['First']))
print('숫자를 글자로 : ',tokenizer.sequences_to_texts([[20,6,9,8,3]]))

글자를 숫자로 :  [[20, 6, 9, 8, 3]]
숫자를 글자로 :  ['f i r s t']


In [ ]:
tokenizer.texts_to_sequences(['abcdet'])

[[5, 22, 19, 13, 2, 3]]

In [ ]:
tokenizer.texts_to_sequences(['\n'])

[[11]]

In [5]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count
print('고유 글자 개수 : ',max_id)
print('전체 글자 개수 : ',dataset_size)

고유 글자 개수 :  39
전체 글자 개수 :  1115394


In [6]:
# 각 글자르 숫자로 인코딩 : 1~39 대신 0~38 얻기 위해 -1 한다.
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_txt]))-1

In [ ]:
encoded[:100]

array([19,  5,  8,  7,  2,  0, 18,  5,  2,  5, 35,  1,  9, 23, 10, 21,  1,
       19,  3,  8,  1,  0, 16,  1,  0, 22,  8,  3, 18,  1,  1, 12,  0,  4,
        9, 15,  0, 19, 13,  8,  2,  6,  1,  8, 17,  0,  6,  1,  4,  8,  0,
       14,  1,  0,  7, 22,  1,  4, 24, 26, 10, 10,  4, 11, 11, 23, 10,  7,
       22,  1,  4, 24, 17,  0,  7, 22,  1,  4, 24, 26, 10, 10, 19,  5,  8,
        7,  2,  0, 18,  5,  2,  5, 35,  1,  9, 23, 10, 15,  3, 13])

## 훈련,검증,테스트 데이터셋
시계열의 데이터는 보통 시간순으로 겹치지 않게 나누는 것이 일반적이다. 암묵적으로 RNN은 과거에서 학습된 패턴이 미래에도 등장한다고 가정한다.<br>
(:=stationary 시계열 데이터가 넓은 의미에서 변하지 않는다 가정)<br>

화학반응의 경우 이런 가정을 적용하기에 무리가 없지만 금융시장의 경우 유의해야한다. 시계열이 진짜로 충분히 안정적인지 확인하려면 시간에 따라 검증 세트에 대한 모델의 오차를 그려볼 수 있다. 검증 세트의 마지막 부분보다 첫 부분에서 성능이 더 좋다면 시계열이 충분히 안정적이지 않다는 의미이므로 더 짧은 시간 간격으로 모델을 훈련하는 것이 좋다.

그렇기 때문에 단순하게 무작위적인 분할을 하는 것은 좋지 않다. 어떤 식으로 데이터 세트를 분할할지 문제에 따라 전략적으로 나누어야 한다.

In [7]:
train_size = round(dataset_size *0.9)
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

# 순차 데이터를 윈도 여러 개로 자르기
훈련 세트는 백만 개 이상의 글자로 이뤄진 시퀀스 하나이다. 여기에 신경망을 직접 훈련 시키기는 어렵다. 이 RNN은 백만 개의 층이 있는 심층 신경망과 비슷하고 (매우 긴) 샘플 하나로 훈련하는 셈이 된다.

대신 데이터셋의 window() 메서드를 사용해 이 긴 시퀀스를 작으면서 많은 텍스트 윈도로 변환한다. 이 데이터셋의 각 샘플은 전체 텍스트에서 매우 짧은 부분 문자열이다. RNN은 이 부분 문자열 길이만큼만 역전파를 위해 펼쳐진다. 이를 TBPTT(truncated backpropagation through time)이라고 부른다. 

In [8]:
n_steps = 100
window_length = n_steps + 1  # target = 1글자 앞의 input
dataset_window = dataset.window(window_length, shift=1, drop_remainder = True)

In [10]:
# pandas의 이동평균 메서드 처럼 첫 번째 윈도는 0~100, 두 번째 윈도는 1~101번 째 글자를 포함하는 식
# 패딩 없이 배치 데이터를 만들기 위해 모든 윈도가 101개로 동일한 글자를 갖기 위해 drop_remainder=True로 지정
# (그렇지 않으면 점점 글자가 줄어 마지막 윈도는 1개의 글자만을 포함할 것)
print(len(dataset),'->',len(dataset_window))

1003855 -> 1003755


window() 메서드는 각각 하나의 데이터셋으로 표현되는 윈도를 포함하는 데이터셋을 만든다.(=리스트의 리스트와 비슷한 중첩 데이터셋(nested dataset)) 이런 구조는 데이터셋 메서드를 호출하여 각 윈도를 변환할 때 유용하다.

하지만 모델은 데이터셋이 아니라 텐서를 기대하므로 훈련에 중첩 데이터셋을 바로 사용하기는 어렵다. 그러므로 플랫 데이터셋으로 변환하는 flat_map() 메서드를 호출해야 한다. flat_map() 메서드는 중첩 데이터셋을 평평하게 만들기 전에 각 데이터셋에 적용할 변환 함수를 매개변수로 받을 수 있다.<br>
ex) lambda ds: ds.batch(2)를 flat_map()에 전달 -> \{\{1,2\},\{3,4,5,6\}\}를 {[1,2],[3,4],[5,6]}으로 변환. 이는 텐서 2개를 가진 데이터셋

In [11]:
dataset_flat = dataset_window.flat_map(lambda window:window.batch(window_length))

이 데이터셋은 연속된 101 글자 길이의 윈도를 담는다.<br>
경사 하강법은 훈련 세트 샘플이 iid일 때 잘 동작하므로 이 윈도를 섞어야 한다.<br>
그런후 윈도를 배치로 만들고 입력(처음 100개의 글자)과 타깃(마지막 글자)를 분리한다. 

In [12]:
batch_size = 32
dataset_shuffle = dataset_flat.shuffle(10000).batch(batch_size)
dataset_shuffle = dataset_shuffle.map(lambda windows : (windows[:,:-1],windows[:,1:]))

In [13]:
dataset_final = dataset_shuffle.map(
    lambda X_batch,Y_batch : (tf.one_hot(X_batch,depth=max_id),Y_batch)
)

In [14]:
dataset_final = dataset_final.prefetch(1)

## Char-RNN 모델 만들고 훈련하기
지나치게 오래 걸리므로 일단 중지

In [16]:
model = keras.models.Sequential([
                                 keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id],
                                                  dropout = 0.2,recurrent_dropout=0.2),
                                 keras.layers.GRU(128,return_sequences=True,dropout=0.2, recurrent_dropout=0.2),
                                 keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))
])

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')
history = model.fit(dataset_final ,epochs=20)

Epoch 1/20
   4838/Unknown - 2760s 570ms/step - loss: 1.6888

KeyboardInterrupt: ignored